In [18]:
import pandas as pd
import requests
import RiotAnalyzer as Ra
from datetime import datetime

In [17]:
api_key_dhil = 'RGAPI-4cc5cb0d-17ed-4196-9bcf-8f9224bad16d'
game_version = "13.17.1"

In [19]:
header = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/116.0.0.0 Safari/537.36",
    "Accept-Language": "en-US,en;q=0.9,id;q=0.8,ja;q=0.7",
    "Accept-Charset": "application/x-www-form-urlencoded; charset=UTF-8",
    "Origin": "https://developer.riotgames.com",
    "X-Riot-Token": f"{api_key_dhil}"
}
url_template = "https://{region_code}.api.riotgames.com/lol/{endpoint}?{query}"

In [20]:
def get_puuid(name:str, region_code:str="NA1"):
    url = url_template.format(region_code=region_code, endpoint="summoner/v4/summoners/by-name/"+name, query="page=1")
    print("PUUID URL: ", url)
    response = requests.get(url, headers=header)
    if response.status_code != 200:
        raise Exception("Error with response code: {}".format(response.status_code))
    else:
        name_data = response.json()
        puuid = name_data["puuid"]
        return puuid

    

In [21]:
def get_mastery(puuid:str, region_code:str="NA1"):
    url = url_template.format(region_code=region_code, endpoint=f"champion-mastery/v4/champion-masteries/by-puuid/{puuid}", query="page=1")
    print("Mastery request URL: ", url)
    response = requests.get(url, headers=header)
    if response.status_code != 200:
        print("Error: ", response.status_code)
    else:
        return response.json()

In [22]:
ra = Ra.RiotAnalyzer(api_key_dhil, version=game_version)

In [23]:
champ_dict = ra.get_champion_dict()

In [24]:
def search_mastery_by_summoner_name(name, region_code:str="NA1"):
    puuid = get_puuid(name, region_code)
    mastery = get_mastery(puuid,region_code)
    df = pd.DataFrame(mastery)
    df["championName"] = df["championId"].apply(lambda x: champ_dict[str(x)])
    df = df[["championName", "championLevel", "championPoints", "lastPlayTime"]]
    df["lastPlayDate"] = df["lastPlayTime"].apply(lambda x: datetime.fromtimestamp(x/1000))
    df["lastPlayTime"] = df["lastPlayTime"].apply(lambda x: datetime.fromtimestamp(x/1000).strftime("%b %d %H:%M"))
    df["percent"] = df["championPoints"]/df["championPoints"].sum()
    df["percent"] = df["percent"].apply(lambda x: "{:.2%}".format(x))
    df.index += 1
    df.index.name = "Rank"
    return df

In [25]:
this = "https://NA1.api.riotgames.com/lol/summoner/v4/summoners/by-name/Michaelhoops?page=1"
that = "https://na1.api.riotgames.com/lol/summoner/v4/summoners/by-name/Michaelhoops?page=1"


In [26]:
df = search_mastery_by_summoner_name("Michaelhoops", "NA1")
df.head(15)

PUUID URL:  https://NA1.api.riotgames.com/lol/summoner/v4/summoners/by-name/Michaelhoops?page=1
Mastery request URL:  https://NA1.api.riotgames.com/lol/champion-mastery/v4/champion-masteries/by-puuid/mw-s09poixSve-gQIt1Jx9fjD0D4P1DVDjmBVwWeS2oMDQPgusVRC0VweeHu0G6dD7NQY5cYcX4v3g?page=1


,championName,championLevel,championPoints,lastPlayTime,lastPlayDate,percent
Rank,,,,,,
1,Shaco,7,250653,Aug 26 13:36,2023-08-26 13:36:47,14.60%
2,Teemo,6,86199,Jul 09 16:04,2023-07-09 16:04:07,5.02%
3,Master Yi,7,81984,May 07 11:55,2022-05-07 11:55:49,4.77%
4,Alistar,7,68186,Sep 10 11:04,2022-09-10 11:04:45,3.97%
5,Shyvana,7,56884,Aug 16 11:07,2022-08-16 11:07:44,3.31%
6,Pantheon,7,54198,Aug 26 14:12,2023-08-26 14:12:23,3.16%
7,Fizz,5,52827,Mar 05 00:43,2022-03-05 00:43:50,3.08%
8,Shen,6,51628,Aug 06 01:02,2023-08-06 01:02:06,3.01%
9,Lux,7,50097,Jul 14 16:43,2023-07-14 16:43:00,2.92%


In [27]:
df[df["championLevel"] == 7]

,championName,championLevel,championPoints,lastPlayTime,lastPlayDate,percent
Rank,,,,,,
1,Shaco,7,250653,Aug 26 13:36,2023-08-26 13:36:47,14.60%
3,Master Yi,7,81984,May 07 11:55,2022-05-07 11:55:49,4.77%
4,Alistar,7,68186,Sep 10 11:04,2022-09-10 11:04:45,3.97%
5,Shyvana,7,56884,Aug 16 11:07,2022-08-16 11:07:44,3.31%
6,Pantheon,7,54198,Aug 26 14:12,2023-08-26 14:12:23,3.16%
9,Lux,7,50097,Jul 14 16:43,2023-07-14 16:43:00,2.92%
10,Thresh,7,43779,Aug 18 22:29,2023-08-18 22:29:04,2.55%


In [28]:
def print_full(x):
    pd.set_option('display.max_rows', 10000)
    display(x)
    pd.reset_option('display.max_rows')

In [29]:
print_full(df)

,championName,championLevel,championPoints,lastPlayTime,lastPlayDate,percent
Rank,,,,,,
1,Shaco,7,250653,Aug 26 13:36,2023-08-26 13:36:47,14.60%
2,Teemo,6,86199,Jul 09 16:04,2023-07-09 16:04:07,5.02%
3,Master Yi,7,81984,May 07 11:55,2022-05-07 11:55:49,4.77%
4,Alistar,7,68186,Sep 10 11:04,2022-09-10 11:04:45,3.97%
5,Shyvana,7,56884,Aug 16 11:07,2022-08-16 11:07:44,3.31%
6,Pantheon,7,54198,Aug 26 14:12,2023-08-26 14:12:23,3.16%
7,Fizz,5,52827,Mar 05 00:43,2022-03-05 00:43:50,3.08%
8,Shen,6,51628,Aug 06 01:02,2023-08-06 01:02:06,3.01%
9,Lux,7,50097,Jul 14 16:43,2023-07-14 16:43:00,2.92%


In [30]:
df


,championName,championLevel,championPoints,lastPlayTime,lastPlayDate,percent
Rank,,,,,,
1,Shaco,7,250653,Aug 26 13:36,2023-08-26 13:36:47,14.60%
2,Teemo,6,86199,Jul 09 16:04,2023-07-09 16:04:07,5.02%
3,Master Yi,7,81984,May 07 11:55,2022-05-07 11:55:49,4.77%
4,Alistar,7,68186,Sep 10 11:04,2022-09-10 11:04:45,3.97%
5,Shyvana,7,56884,Aug 16 11:07,2022-08-16 11:07:44,3.31%
...,...,...,...,...,...,...
140,Diana,1,123,Apr 16 12:24,2022-04-16 12:24:36,0.01%
141,Vayne,1,123,Mar 05 22:22,2021-03-05 22:22:59,0.01%
142,Gangplank,1,114,Apr 10 02:26,2022-04-10 02:26:01,0.01%


In [31]:
def get_champion_dict(version="13.17.1"):
    """Gets a dictionary of champion names and their IDs

    Args:
        version (str): The version of the game to get the champion data for

    Returns:
        dict: A dictionary of champion names and their IDs
    """
    url = f"http://ddragon.leagueoflegends.com/cdn/{version}/data/en_US/champion.json"
    response = requests.get(url)
    if response.status_code != 200:
        raise Exception(f"Error {response.status_code} when querying champion data")
    data = response.json()
    # Take data and put inside a pandas dataframe
    df = pd.DataFrame(data["data"]).T
    return df

In [ ]:
cd = get_champion_dict()
cd.get('key')['Aatrox']

In [ ]:
cd.loc["Neeko"]

In [ ]:
cd.loc["Vex"]['tags']

In [ ]:
cd.to_csv('biden.csv', index=False, na_rep = 'biden')

In [ ]:
cr = {}
cr['Aatrox'] = [1]
cr['Ahri'] = [3]
cr

In [ ]:
api_key_dhil = 'RGAPI-4cc5cb0d-17ed-4196-9bcf-8f9224bad16d'
game_version = "13.17.1"

In [1]:
def get_puuid(name:str, region_code:str="NA1"):
    url = url_template.format(region_code=region_code, endpoint="summoner/v4/summoners/by-name/"+name, query="page=1")
    print("PUUID URL: ", url)
    response = requests.get(url, headers=header)
    if response.status_code != 200:
        raise Exception("Error with response code: {}".format(response.status_code))
    else:
        name_data = response.json()
        puuid = name_data["puuid"]
        return puuid

    

In [2]:
ra = Ra.RiotAnalyzer(api_key_dhil, version=game_version)

NameError: name 'Ra' is not defined

In [ ]:
def search_mastery_by_summoner_name(name, region_code:str="NA1"):
    puuid = get_puuid(name, region_code)
    mastery = get_mastery(puuid,region_code)
    df = pd.DataFrame(mastery)
    df["championName"] = df["championId"].apply(lambda x: champ_dict[str(x)])
    df = df[["championName", "championLevel", "championPoints", "lastPlayTime"]]
    df["lastPlayDate"] = df["lastPlayTime"].apply(lambda x: datetime.fromtimestamp(x/1000))
    df["lastPlayTime"] = df["lastPlayTime"].apply(lambda x: datetime.fromtimestamp(x/1000).strftime("%b %d %H:%M"))
    df["percent"] = df["championPoints"]/df["championPoints"].sum()
    df["percent"] = df["percent"].apply(lambda x: "{:.2%}".format(x))
    df.index += 1
    df.index.name = "Rank"
    return df

In [ ]:
df = search_mastery_by_summoner_name("Michaelhoops", "NA1")
df.head(15)

PUUID URL:  https://NA1.api.riotgames.com/lol/summoner/v4/summoners/by-name/Michaelhoops?page=1


Mastery request URL:  https://NA1.api.riotgames.com/lol/champion-mastery/v4/champion-masteries/by-puuid/mw-s09poixSve-gQIt1Jx9fjD0D4P1DVDjmBVwWeS2oMDQPgusVRC0VweeHu0G6dD7NQY5cYcX4v3g?page=1


,championName,championLevel,championPoints,lastPlayTime,lastPlayDate,percent
Rank,,,,,,
1,Shaco,7,250653,Aug 26 13:36,2023-08-26 13:36:47,14.60%
2,Teemo,6,86199,Jul 09 16:04,2023-07-09 16:04:07,5.02%
3,Master Yi,7,81984,May 07 11:55,2022-05-07 11:55:49,4.77%
4,Alistar,7,68186,Sep 10 11:04,2022-09-10 11:04:45,3.97%
5,Shyvana,7,56884,Aug 16 11:07,2022-08-16 11:07:44,3.31%
6,Pantheon,7,54198,Aug 26 14:12,2023-08-26 14:12:23,3.16%
7,Fizz,5,52827,Mar 05 00:43,2022-03-05 00:43:50,3.08%
8,Shen,6,51628,Aug 06 01:02,2023-08-06 01:02:06,3.01%
9,Lux,7,50097,Jul 14 16:43,2023-07-14 16:43:00,2.92%


In [ ]:
def print_full(x):
    pd.set_option('display.max_rows', 10000)
    display(x)
    pd.reset_option('display.max_rows')

In [ ]:
df


,championName,championLevel,championPoints,lastPlayTime,lastPlayDate,percent
Rank,,,,,,
1,Shaco,7,250653,Aug 26 13:36,2023-08-26 13:36:47,14.60%
2,Teemo,6,86199,Jul 09 16:04,2023-07-09 16:04:07,5.02%
3,Master Yi,7,81984,May 07 11:55,2022-05-07 11:55:49,4.77%
4,Alistar,7,68186,Sep 10 11:04,2022-09-10 11:04:45,3.97%
5,Shyvana,7,56884,Aug 16 11:07,2022-08-16 11:07:44,3.31%
...,...,...,...,...,...,...
140,Diana,1,123,Apr 16 12:24,2022-04-16 12:24:36,0.01%
141,Vayne,1,123,Mar 05 22:22,2021-03-05 22:22:59,0.01%
142,Gangplank,1,114,Apr 10 02:26,2022-04-10 02:26:01,0.01%


In [42]:
cd = get_champion_dict()
cd.get('key')['Aatrox']

'266'

In [44]:
cd.loc["Vex"]['tags']

['Mage']

In [46]:
cr = {}
cr['Aatrox'] = [1]
cr['Ahri'] = [3]
cr

{'Aatrox': [1], 'Ahri': [3]}